In [330]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum

from blowfish import Blowfish

In [331]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [332]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [333]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [334]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [335]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [336]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [337]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [338]:
_media_file_stream = open(media_pk2_path, 'rb')

In [339]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [340]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [341]:
blowfish_checksum_decoded = "Joymax Pak File"

In [342]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [343]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [344]:
Root = read_blocks_at(256)

In [345]:
blocks_in_memory = {
  "": Root
}

In [346]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

## DIVISIONINFO.TXT

In [347]:
division_info_txt = "DIVISIONINFO.TXT"

In [348]:
root = blocks_in_memory[""]
entry = None
for entries in root:
  entry = next((x for x in entries["Entries"] if x["Name"] == division_info_txt), None)
  if entry:
    break

entry

{'Type': <PackEntryType.File: 2>,
 'Name': 'DIVISIONINFO.TXT',
 'CreationTime': 133336210349991177,
 'ModifyTime': 133336210349991177,
 'DataPosition': 1017319424,
 'DataSize': 57,
 'NextBlock': 0,
 'Payload': b'\x00\x00'}

In [349]:
_media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"]))

Locale = buffer.read(1)[0]
division_count = buffer.read(1)[0]

for i in range(division_count):
  length = struct.unpack('<i', buffer.read(4))[0]
  division_name = buffer.read(length).decode('utf-8')
  print(division_name)
  buffer.read(1) # Skip null terminator
  gateway_count = buffer.read(1)[0]
  for j in range(gateway_count):
    length = struct.unpack('<i', buffer.read(4))[0]
    gateway_name = buffer.read(length).decode('utf-8')
    print(gateway_name)
    buffer.read(1)

DIV01
gw1sro.vtcgame.vn
gw2sro.vtcgame.vn


## GATEPORT.TXT

In [350]:
gate_port_txt = "GATEPORT.TXT"

In [351]:
root = blocks_in_memory[""]
entry = None
for entries in root:
  entry = next((x for x in entries["Entries"] if x["Name"] == gate_port_txt), None)
  if entry:
    break

entry

{'Type': <PackEntryType.File: 2>,
 'Name': 'GATEPORT.TXT',
 'CreationTime': 133336210351001190,
 'ModifyTime': 133336210351001190,
 'DataPosition': 1017323065,
 'DataSize': 8,
 'NextBlock': 0,
 'Payload': b'\x00\x00'}

In [352]:
_media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
buffer = _media_file_stream.read(entry["DataSize"])

In [353]:
port = int(buffer.decode('ascii').rstrip('\0'))
port

15779

## SV.T

In [354]:
sv_t = "SV.T"

In [355]:
root = blocks_in_memory[""]
entry = None
for entries in root:
  entry = next((x for x in entries["Entries"] if x["Name"] == sv_t), None)
  if entry:
    break

entry

{'Type': <PackEntryType.File: 2>,
 'Name': 'SV.T',
 'CreationTime': 133795825379609632,
 'ModifyTime': 133795825379609632,
 'DataPosition': 1017319481,
 'DataSize': 1024,
 'NextBlock': 1017320505,
 'Payload': b'\x00\x00'}

In [356]:
_media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"]))

In [357]:
version_buffer_length = struct.unpack('<i', buffer.read(4))[0]
version_buffer = buffer.read(version_buffer_length)

In [358]:
key = "SILKROADVERSION"
key_buffer = key.encode('ascii')[:8]
sv_blowfish = Blowfish()
sv_blowfish.Initialize(key_buffer)

In [359]:
decoded_version_buffer = sv_blowfish.Decode(version_buffer)[:4]
version = int(decoded_version_buffer.decode('ascii').rstrip('\0'))
version

15